In [2]:
import numpy as np
import pandas as pd
import networkx as nx

In [58]:
#filepath = '../datasets/p_0_1_percent/PS_20230505183351_53313/PS_20230505183351_53313_rawLog.csv' # p = 0.1%
#filepath = '../datasets/p_0_5_percent/PS_20230509175835_53313/PS_20230509175835_53313_rawLog.csv' # p = 0.5%
#filepath = '../datasets/p_1_percent/PS_20230509180228_53313/PS_20230509180228_53313_rawLog.csv' # p = 1%
filepath = '../datasets/p_5_percent/PS_20230509180753_53313/PS_20230509180753_53313_rawLog.csv' # p = 5%

df = pd.read_csv(filepath)

In [59]:
source = df["nameOrig"].unique()
target = df["nameDest"].unique()
c = set(source).union(set(target))
len(c), len(set(source)), len(set(target))
accounts = list(c)
accounts_df = pd.DataFrame(accounts,columns=['account'])

In [60]:

# Create a directed graph
graph_orig = nx.MultiDiGraph()   
graph2 = nx.DiGraph()
graph3 = nx.DiGraph()

# Add nodes to the graph using the accounts DataFrame
for index, row in accounts_df.iterrows():
    graph_orig.add_node(row["account"])
    graph2.add_node(row["account"])
    graph3.add_node(row["account"])

# Add edges to the graph using the transactions DataFrame
for index, row in df.iterrows():
    graph_orig.add_edge(row["nameOrig"], row["nameDest"], action=row["action"], amount=row["amount"], oldBalanceOrig=row["oldBalanceOrig"], newBalanceOrig=row["newBalanceOrig"], oldBalanceDest=row["oldBalanceDest"], newBalanceDest=row["newBalanceDest"], isFraud=row["isFraud"], isFlaggedFraud=row["isFlaggedFraud"], isUnauthorizedOverdraft=row["isUnauthorizedOverdraft"])


In [61]:
accounts_np = np.empty((0,44))

count = 0
for node in graph_orig.nodes: #loop through nodes. Current node is u
    '''
    if count > 100:
        break
    count += 1
    '''

    incoming_edges = graph_orig.in_edges(node, data=True)

    actions_in_list_arr = np.empty((0, 5))
    actions_dependent_in_function_arr = np.empty((0, 5))
    balance_arr = np.array([])
    balance_new_arr = np.array([])
    balance_old_arr = np.array([])
    amounts_in_arr = np.array([])
    countains_flagged_fraud_arr = np.array([])
    countains_unauth_overdraft_arr = np.array([])
    countains_in_fraud_arr = np.array([])

    node_in_max_cash_out = 0
    node_in_max_transfer = 0
    
    for source, target, edge_data in incoming_edges:
        action = edge_data['action']
        
        actions_in_list = np.zeros(5)
        actions_dependent_in_function = np.zeros(5)
        if action == 'TRANSFER':
            node_in_max_transfer = np.max([node_in_max_transfer,edge_data['amount']])
            actions_in_list[0] = 1
            if np.max([edge_data['oldBalanceDest'], edge_data['newBalanceDest']]) == 0:
                actions_dependent_in_function[0] = 1 #IMPORTANT BECAUSE THIS IS CAUSED BY 0.0 / 0.0 WHICH WE WANT TO REMAIN HIGHT
            else:
                actions_dependent_in_function[0] = np.divide(np.min([edge_data['oldBalanceDest'], edge_data['newBalanceDest']]), np.max([edge_data['oldBalanceDest'], edge_data['newBalanceDest']]))
        elif action == 'CASH_IN':
            actions_in_list[1] = 1
            if np.max([edge_data['oldBalanceDest'], edge_data['newBalanceDest']]) == 0:
                actions_dependent_in_function[1] = 1 #IMPORTANT BECAUSE THIS IS CAUSED BY 0.0 / 0.0 WHICH WE WANT TO REMAIN HIGHT
            else:
                actions_dependent_in_function[1] = np.divide(np.min([edge_data['oldBalanceDest'], edge_data['newBalanceDest']]) , np.max([edge_data['oldBalanceDest'], edge_data['newBalanceDest']]))            
        elif action == 'CASH_OUT':
            node_in_max_cash_out = np.max([node_in_max_cash_out,edge_data['amount']])
            actions_in_list[2] = 1
            if np.max([edge_data['oldBalanceDest'], edge_data['newBalanceDest']]) == 0:
                actions_dependent_in_function[2] = 1 #IMPORTANT BECAUSE THIS IS CAUSED BY 0.0 / 0.0 WHICH WE WANT TO REMAIN HIGHT
            else:
                actions_dependent_in_function[2] = np.divide(np.min([edge_data['oldBalanceDest'], edge_data['newBalanceDest']]) , np.max([edge_data['oldBalanceDest'], edge_data['newBalanceDest']]))
        elif action == 'DEBIT':
            actions_in_list[3] = 1
            if np.max([edge_data['oldBalanceDest'], edge_data['newBalanceDest']]) == 0:
                actions_dependent_in_function[3] = 1 #IMPORTANT BECAUSE THIS IS CAUSED BY 0.0 / 0.0 WHICH WE WANT TO REMAIN HIGHT
            else:
                actions_dependent_in_function[3] = np.divide(np.min([edge_data['oldBalanceDest'], edge_data['newBalanceDest']]) , np.max([edge_data['oldBalanceDest'], edge_data['newBalanceDest']]))
        elif action == 'PAYMENT':
            actions_in_list[4] = 1
            if np.max([edge_data['oldBalanceDest'], edge_data['newBalanceDest']]) == 0:
                actions_dependent_in_function[4] = 1 #IMPORTANT BECAUSE THIS IS CAUSED BY 0.0 / 0.0 WHICH WE WANT TO REMAIN HIGHT
            else:
                actions_dependent_in_function[4] = np.divide(np.min([edge_data['oldBalanceDest'], edge_data['newBalanceDest']]) , np.max([edge_data['oldBalanceDest'], edge_data['newBalanceDest']]))
        
        actions_in_list_arr = np.append(actions_in_list_arr, [actions_in_list], axis=0)
        actions_dependent_in_function_arr = np.append(actions_dependent_in_function_arr, [actions_dependent_in_function], axis=0)
            
        balance_arr = np.append(balance_arr, edge_data['newBalanceDest'])
        balance_arr = np.append(balance_arr, edge_data['oldBalanceDest'])

        amounts_in_arr = np.append(amounts_in_arr, edge_data['amount'])

        countains_flagged_fraud_arr = np.append(countains_flagged_fraud_arr, edge_data['isFlaggedFraud'])
        countains_unauth_overdraft_arr = np.append(countains_unauth_overdraft_arr, edge_data['isUnauthorizedOverdraft'])

        countains_in_fraud_arr = np.append(countains_in_fraud_arr, edge_data['isFraud'])

    
    outgoing_edges = graph_orig.out_edges(node, data=True)

    node_out_max_cash_out = 0
    node_out_max_transfer = 0

    actions_out_list_arr = np.empty((0, 5))
    actions_dependent_out_function_arr = np.empty((0, 5))
    countains_out_fraud_arr = np.array([])
    amounts_out_arr = np.array([])
    for source, target, edge_data in outgoing_edges:
        #amounts_out_arr = np.append(amounts_out_arr, edge_data['amount'])
        
        actions_out_list = np.zeros(5)
        actions_dependent_out_function = np.zeros(5)
        if action == 'TRANSFER':
            node_out_max_transfer = np.max([node_out_max_transfer, edge_data['amount']])
            actions_out_list[0] = 1
            if np.max([edge_data['oldBalanceOrig'], edge_data['newBalanceOrig']]) == 0:
                actions_dependent_in_function[0] = 1 #IMPORTANT BECAUSE THIS IS CAUSED BY 0.0 / 0.0 WHICH WE WANT TO REMAIN HIGHT
            else:
                actions_dependent_out_function[0] = np.min([edge_data['oldBalanceOrig'], edge_data['newBalanceOrig']]) / np.max([edge_data['oldBalanceOrig'], edge_data['newBalanceOrig']])
        elif action == 'CASH_IN':
            actions_out_list[1] = 1
            if np.max([edge_data['oldBalanceOrig'], edge_data['newBalanceOrig']]) == 0:
                actions_dependent_in_function[1] = 1 #IMPORTANT BECAUSE THIS IS CAUSED BY 0.0 / 0.0 WHICH WE WANT TO REMAIN HIGHT
            else:
                actions_dependent_out_function[1] = np.min([edge_data['oldBalanceOrig'], edge_data['newBalanceOrig']]) / np.max([edge_data['oldBalanceOrig'], edge_data['newBalanceOrig']])
        elif action == 'CASH_OUT':
            node_out_max_cash_out = np.max([node_out_max_cash_out, edge_data['amount']])
            actions_out_list[2] = 1
            if np.max([edge_data['oldBalanceOrig'], edge_data['newBalanceOrig']]) == 0:
                actions_dependent_in_function[2] = 1 #IMPORTANT BECAUSE THIS IS CAUSED BY 0.0 / 0.0 WHICH WE WANT TO REMAIN HIGHT
            else:
                actions_dependent_out_function[2] = np.min([edge_data['oldBalanceOrig'], edge_data['newBalanceOrig']]) / np.max([edge_data['oldBalanceOrig'], edge_data['newBalanceOrig']])
        elif action == 'DEBIT':
            actions_out_list[3] = 1
            if np.max([edge_data['oldBalanceOrig'], edge_data['newBalanceOrig']]) == 0:
                actions_dependent_in_function[3] = 1 #IMPORTANT BECAUSE THIS IS CAUSED BY 0.0 / 0.0 WHICH WE WANT TO REMAIN HIGHT
            else:
                actions_dependent_out_function[3] = np.min([edge_data['oldBalanceOrig'], edge_data['newBalanceOrig']]) / np.max([edge_data['oldBalanceOrig'], edge_data['newBalanceOrig']])
        elif action == 'PAYMENT':
            actions_out_list[4] = 1
            if np.max([edge_data['oldBalanceOrig'], edge_data['newBalanceOrig']]) == 0:
                actions_dependent_in_function[4] = 1 #IMPORTANT BECAUSE THIS IS CAUSED BY 0.0 / 0.0 WHICH WE WANT TO REMAIN HIGHT
            else:
                actions_dependent_out_function[4] = np.min([edge_data['oldBalanceOrig'], edge_data['newBalanceOrig']]) / np.max([edge_data['oldBalanceOrig'], edge_data['newBalanceOrig']])
        
        actions_dependent_out_function_arr = np.append(actions_dependent_out_function_arr, [actions_dependent_out_function], axis=0)

        actions_out_list_arr = np.append(actions_out_list_arr, [actions_out_list], axis=0)

        countains_out_fraud_arr = np.append(countains_out_fraud_arr, edge_data['isFraud'])

    if balance_arr.size == 0:
        balance_arr = np.append(balance_arr, 0)

    if balance_new_arr.size == 0:
        balance_new_arr = np.append(balance_new_arr, 0)

    if balance_old_arr.size == 0:
        balance_old_arr = np.append(balance_old_arr, 0)

    if actions_in_list_arr.size == 0:
        actions_in_list_arr = np.zeros(5)[np.newaxis]
    
    if actions_out_list_arr.size == 0:
        actions_out_list_arr = np.zeros(5)[np.newaxis]

    if actions_dependent_in_function_arr.size == 0:
        actions_dependent_in_function_arr = np.zeros(5)[np.newaxis]

    if actions_dependent_out_function_arr.size == 0:
        actions_dependent_out_function_arr = np.zeros(5)[np.newaxis]


    if countains_flagged_fraud_arr.size == 0:
        countains_flagged_fraud_arr = np.append(countains_flagged_fraud_arr, 0)

    if countains_unauth_overdraft_arr.size == 0:
        countains_unauth_overdraft_arr = np.append(countains_unauth_overdraft_arr, 0)

    if countains_in_fraud_arr.size == 0:
        countains_in_fraud_arr = np.append(countains_in_fraud_arr, 0)

    if countains_out_fraud_arr.size == 0:
        countains_out_fraud_arr = np.append(countains_out_fraud_arr, 0)

    if amounts_in_arr.size == 0:
        amounts_in_arr = np.append(amounts_in_arr,0)

    if amounts_out_arr.size == 0:
        amounts_out_arr = np.append(amounts_in_arr,0)
    

    node_deg_in = len(incoming_edges)
    node_deg_out = len(outgoing_edges)
    node_deg_total = node_deg_in + node_deg_out

    node_min_balance = np.min(balance_arr)
    node_max_balance = np.max(balance_arr)

    node_min_new_balance = np.min(balance_new_arr) #MAYBE REMOVE
    node_max_new_balance = np.max(balance_new_arr) #MAYBE REMOVE

    node_min_old_balance = np.min(balance_old_arr) #MAYBE REMOVE
    node_max_old_balance = np.max(balance_old_arr) #MAYBE REMOVE

    node_mean_new_balance = np.mean(balance_new_arr)
    node_mean_old_balance = np.mean(balance_old_arr)

    node_std_new_balance = np.std(balance_new_arr)
    node_std_old_balance = np.std(balance_old_arr)

    node_median_new_balance = np.median(balance_new_arr)
    node_median_old_balance = np.median(balance_old_arr)

    node_mean_in_amount = np.mean(amounts_in_arr)
    node_mean_out_amount = np.mean(amounts_out_arr)

    node_std_in_amount = np.std(amounts_in_arr)
    node_std_out_amount = np.std(amounts_out_arr)

    node_median_in_amount = np.median(amounts_in_arr)
    node_median_out_amount = np.median(amounts_out_arr)

    node_action_in = np.max(actions_in_list_arr, axis=0)
    node_actions_dependent_in_function = np.max(actions_dependent_in_function_arr, axis=0)

    node_action_out = np.max(actions_out_list_arr, axis=0)
    node_actions_dependent_out_function = np.max(actions_dependent_out_function_arr, axis=0)

    node_is_merchant = 0
    node_is_client = 0
    
    if node[0] == "M":
        node_is_merchant = 1
    if node[0] == "C":
        node_is_client = 1
    

    node_countains_flagged_fraud = np.max(countains_flagged_fraud_arr)
    node_countains_unauth_overdraft = np.max(countains_unauth_overdraft_arr)

    node_in_isFraud = np.max(countains_in_fraud_arr)
    node_out_isFraud = np.max(countains_out_fraud_arr)

    node_isFraud = 0
    if node_in_isFraud == 1 and node_out_isFraud == 1 and node[0:2] == "CC":
        node_isFraud = 1
    
    '''
    print(np.array([node_deg_in, node_deg_out, node_deg_total, node_min_balance, node_max_balance, node_min_new_balance, node_max_new_balance, \
                                      node_mean_new_balance, node_mean_old_balance, node_std_new_balance, node_std_old_balance, node_median_in_amount, \
                                      node_median_out_amount]))
    print(node_actions,actions_list_arr )
    print(node_actions_dependent_in_function,actions_dependent_in_function_arr)
    print(np.array([node_countains_flagged_fraud, \
                                      node_countains_unauth_overdraft, node_in_isFraud, node_out_isFraud]))
    '''
                                      
    node_attributes = np.concatenate((np.array([node_deg_in, node_deg_out, node_deg_total, node_min_balance, node_max_balance, node_min_new_balance, node_max_new_balance, \
                                      node_mean_new_balance, node_mean_old_balance, node_std_new_balance, node_std_old_balance, node_median_in_amount, \
                                      node_median_out_amount, node_is_merchant, node_is_client, node_out_max_transfer, node_out_max_cash_out, node_in_max_transfer, node_in_max_cash_out]), \
                                      node_action_in, node_actions_dependent_in_function, node_action_out, node_actions_dependent_out_function, np.array([node_countains_flagged_fraud, \
                                      node_countains_unauth_overdraft, node_in_isFraud, node_out_isFraud, node_isFraud])))

    accounts_np = np.append(accounts_np,[node_attributes],axis=0)
    

    #print(len(node_attributes))
    #print("NEXT NODE")
    

In [62]:
column_names = ['node_deg_in','node_deg_out','node_deg_total','node_min_balance','node_max_balance','node_min_new_balance','node_max_new_balance','node_mean_new_balance',\
                'node_mean_old_balance','node_std_new_balance','node_std_old_balance','node_median_in_amount','node_median_out_amount','node_is_merchant','node_is_client',\
                'node_out_max_transfer','node_out_max_cash_out','node_in_max_transfer','node_in_max_cash_out','node_action_in_TRANSFER','node_action_in_CASHIN',\
                'node_action_in_CASHOUT','node_action_in_DEBIT','node_action_in_PAYMENT','node_action_in_dependent_in_function_TRANSFER','node_actions_dependent_in_function_CASHOIN',\
                'node_actions_dependent_in_function_CASHOUT','node_actions_dependent_in_function_DEBIT','node_actions_dependent_in_function_PAYMENT', 'node_action_out_TRANSFER',\
                'node_action_out_CASHIN','node_action_out_CASHOUT','node_action_out_DEBIT','node_action_out_PAYMENT','node_actions_dependent_out_function_TRANSFER',\
                'node_actions_dependent_out_function_CASHIN','node_actions_dependent_out_function_CASHOUT','node_actions_dependent_out_function_DEBIT','node_actions_dependent_out_function_PAYMENT',\
                'node_countains_flagged_fraud','node_countains_unauth_overdraft','node_in_isFraud','node_out_isFraud','node_isFraud']

accounts_df = pd.DataFrame(accounts_np, columns=column_names)

In [63]:
accounts_df.head()

,node_deg_in,node_deg_out,node_deg_total,node_min_balance,node_max_balance,node_min_new_balance,node_max_new_balance,node_mean_new_balance,node_mean_old_balance,node_std_new_balance,...,node_actions_dependent_out_function_TRANSFER,node_actions_dependent_out_function_CASHIN,node_actions_dependent_out_function_CASHOUT,node_actions_dependent_out_function_DEBIT,node_actions_dependent_out_function_PAYMENT,node_countains_flagged_fraud,node_countains_unauth_overdraft,node_in_isFraud,node_out_isFraud,node_isFraud
0,16.0,90.0,106.0,679490.19,5574983.18,0.0,0.0,0.0,0.0,0.0,...,0.999965,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
1,1.0,1.0,2.0,0.00,73807.50,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0
2,1.0,1.0,2.0,0.00,3612227.71,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0
3,6.0,10.0,16.0,-12939.64,2211504.43,0.0,0.0,0.0,0.0,0.0,...,0.998022,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,78.0,0.0,78.0,0.00,103766.42,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [64]:
accounts_df.to_csv(filepath[0:-10]+'account_attributes.csv',index=0)